In [1]:
#import libraries 
import gym
from env.custom_hopper import *
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor

import numpy as np
import matplotlib.pyplot as plt

/home/federica/anaconda3/envs/mldl/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env = gym.make('CustomHopper-source-v0')


In [3]:
print(f"default mass of the torso:{env.sim.model.body_mass[1]} \ndefault mass of the thigh: {env.sim.model.body_mass[2]} \ndefault mass of the leg: {env.sim.model.body_mass[3]} \ndefault mass of the foot: {env.sim.model.body_mass[4]}")


default mass of the torso:2.534291735288517 
default mass of the thigh: 3.9269908169872427 
default mass of the leg: 2.7143360527015816 
default mass of the foot: 5.0893800988154645


Define initial and current mass range for the thigh, leg, and foot. We keep the mass of the torso fixed at its default value.

In [3]:
randomization_params = {
    'thigh_mass': {'initial_range': [3.5, 4.5], 'current_range': [3.5, 4.5]},
    'leg_mass': {'initial_range': [2.25, 3.25], 'current_range': [2.25, 3.25]},
    'foot_mass': {'initial_range': [4.75, 5.25], 'current_range': [4.75, 5.25]}
}

Create an AutoDR wrapper that apply parameters randomization

In [6]:
class AutoDRWrapper(gym.Wrapper):
    def __init__(self, env, randomization_params):
        super().__init__(env)
        self.randomization_params = randomization_params
    
    def reset(self):
        self.apply_randomization()
        return self.env.reset()
    
    def apply_randomization(self):
        for param, range_dict in self.randomization_params.items():
            value = np.random.uniform(*range_dict['current_range'])
            self.set_env_param(param, value)
    
    def set_env_param(self, param, value):
        if param == 'thigh_mass':
            self.env.model.body_mass[2] = value
        elif param == 'leg_mass':
            self.env.model.body_mass[3] = value
        elif param == 'foot_mass':
            self.env.model.body_mass[4] = value

    def expand_boundaries(self, performance, target_performance, expansion_rate):
        for param, range_dict in self.randomization_params.items():
            lower, upper = range_dict['current_range']
            center = (lower + upper) / 2
            if performance >= target_performance:
                new_lower = center - (center - lower) * (1 + expansion_rate)
                new_upper = center + (upper - center) * (1 + expansion_rate)
            else:
                new_lower = center - (center - lower) / (1 + expansion_rate)
                new_upper = center + (upper - center) / (1 + expansion_rate)
            
            range_dict['current_range'] = [new_lower, new_upper]

    def train_with_autodr(env, agent, num_episodes, target_performance, expansion_rate):
        performance_history = []
        
        for episode in range(num_episodes):
            state = env.reset()
            episode_reward = 0
            done = False
            while not done:
                action = agent.predict(state)
                next_state, reward, done, _ = env.step(action)
                #agent.learn(state, action, reward, next_state, done)
                agent.learn(total_timesteps=1)
                state = next_state
                episode_reward += reward
            
            # Update randomization boundaries
            env.expand_boundaries(episode_reward, target_performance, expansion_rate)
            performance_history.append(episode_reward)
            
            # Log progress
            if episode % 100 == 0:
                print(f"Episode {episode}, Reward: {episode_reward}")
                for param, range_dict in env.randomization_params.items():
                    print(f"  {param}: {range_dict['current_range']}")


TO ADJUST: train_with_autodr has to be outside the class
The expand_boundaries function adjusts the randomization ranges based on the agent's performance.

In [7]:
def train_with_autodr(env, agent, num_episodes, target_performance, expansion_rate):
    performance_history = []
    
    for episode in range(num_episodes):
        state = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action = agent.predict(state)
            next_state, reward, done, _ = env.step(action)
            #agent.learn(state, action, reward, next_state, done)
            agent.learn(total_timesteps=1)
            state = next_state
            episode_reward += reward
        
        # Update randomization boundaries
        env.envs[0].expand_boundaries(episode_reward, target_performance, expansion_rate)
        performance_history.append(episode_reward)
        
        # Log progress
        if episode % 100 == 0:
            print(f"Episode {episode}, Reward: {episode_reward}")
            for param, range_dict in env.envs[0].randomization_params.items():
                print(f"  {param}: {range_dict['current_range']}")

In [8]:
# Step 2: Create and train PPO agent
# Wrap the environment
myenv = DummyVecEnv([lambda: AutoDRWrapper(env, randomization_params)])

# Create the agent
agent = PPO("MlpPolicy", myenv, verbose=1)

# Train the agent
num_episodes = 1000
target_performance = 100
expansion_rate = 0.1

performance_history = train_with_autodr(myenv, agent, num_episodes, target_performance, expansion_rate)


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 234  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 251  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 426  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 474  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------
| time/              |      |
|    fps             | 599  |
|    iterations      | 1    |
|    time_elapsed    | 

MujocoException: Got MuJoCo Warning: Linesearch objective is not convex

## ChatGPT explanation for the Mujoco warning:
The warning `MujocoException: Got MuJoCo Warning: Linesearch objective is not convex` occurs during the optimization process within the MuJoCo physics engine, specifically when MuJoCo tries to find a solution during a linesearch, a step in numerical optimization algorithms. This warning suggests that the objective function being optimized (often related to forces, torques, or positions) is not convex, meaning it doesn't have a single, well-defined minimum, making it difficult for the optimizer to converge reliably.

Given the context of your code, where you are randomizing the masses of the `thigh`, `leg`, and `foot` in your `CustomHopper-source-v0` environment, the warning likely arises due to the physical properties of your simulated environment being altered in such a way that the dynamics become more complex or unstable.

### Possible Reasons for the Warning:

1. **Nonlinear Dynamics**: Changing the masses of body parts can introduce nonlinear dynamics into the system. The hopper might behave in a less predictable way, making the optimization landscape (which the physics engine is trying to navigate) more complex and possibly non-convex.

2. **Mass Values Causing Instability**: If the randomized mass values are too far from their original values, the robot might become unstable, making it hard for the physics engine to find stable configurations. This could lead to situations where small changes in state lead to large, unpredictable changes in the objective function.

3. **Solver Difficulties**: The MuJoCo engine uses a solver to compute physics interactions. If the physical parameters (like mass) make the system too stiff or too soft, the solver may struggle to find a stable solution, especially if the objective function isn't smooth or well-behaved.

### Addressing the Issue:

- **Tighten the Mass Ranges**: You might want to reduce the range of randomization for the masses to avoid extreme values that could cause instability.

- **Monitor Simulation Stability**: Keep an eye on the hopper's behavior during training. If the hopper exhibits erratic or unnatural movements, consider adjusting the randomization parameters or the physical properties like damping, friction, or joint stiffness.

- **Debug with Smaller Changes**: Try incrementally changing the masses rather than allowing a wide range of values all at once. This can help you identify a threshold where the simulation starts to become unstable.

- **Adjust MuJoCo Parameters**: If possible, tweaking MuJoCo’s solver parameters (e.g., increasing the number of solver iterations or adjusting the tolerance) might help the engine handle more complex dynamics better.

This warning is an indication that the physics simulation is having trouble due to the altered mass properties, and addressing the issue will likely involve tuning the randomization process or the simulation parameters.

In [ ]:
# Step 5: Plot the performance
plt.figure(figsize=(10, 6))
plt.plot(performance_history)
plt.title('Agent Performance over Episodes')
plt.xlabel('Episode')
plt.ylabel('Performance')
plt.show()

# Save the trained model
agent.save("ppo_autodr")